# 磁盘信息

> 磁盘的监控还是相当重要的, 磁盘满了程序异常随之而来. 监控策略的话根据`percent`来进行告警.  监控时将磁盘详细信息采集下来可以更好的了解磁盘使用状况.

In [2]:
# %load ../util.py
def bytes2human(n):
    # http://code.activestate.com/recipes/578019
    # >>> bytes2human(10000)
    # '9.8K'
    # >>> bytes2human(100001221)
    # '95.4M'
    symbols = ('K', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    prefix = {}
    for i, s in enumerate(symbols):
        prefix[s] = 1 << (i + 1) * 10
    for s in reversed(symbols):
        if n >= prefix[s]:
            value = float(n) / prefix[s]
            return '%.1f%s' % (value, s)
    return "%sB" % n


In [7]:
# 类似于df命令
import os
import psutil

templ = "%-30s %8s %8s %8s %5s%% %9s  %s"
print(templ % ("Device", "Total", "Used", "Free", "Use ", "Type",
               "Mount"))
for part in psutil.disk_partitions(all=False):
    if os.name == 'nt':
        if 'cdrom' in part.opts or part.fstype == '':
            # skip cd-rom drives with no disk in it; they may raise
            # ENOENT, pop-up a Windows GUI error for a non-ready
            # partition or just hang.
            continue
    usage = psutil.disk_usage(part.mountpoint)
    print(templ % (
        part.device,
        bytes2human(usage.total),
        bytes2human(usage.used),
        bytes2human(usage.free),
        int(usage.percent),
        part.fstype,
        part.mountpoint))


Device                            Total     Used     Free  Use %      Type  Mount
/dev/mapper/vagrant--vg-root      18.3G    13.7G     3.6G    79%      ext4  /
/dev/sda1                        235.3M   177.8M    45.4M    79%      ext2  /boot
